In [1]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 24.7 MB/s eta 0:00:00


In [2]:
!apt update
!apt install -y chromium-browser chromium-chromedriver
!pip install selenium webdriver-manager
!which chromedriver  # Check if Chromedriver is installed


Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,692 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,679 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Pac

In [3]:
!pip install selenium webdriver-manager pandas


In [ ]:
import subprocess
import sys

# Install dependencies
def install_requirements():
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-r", "requirements.txt"])

if __name__ == "__main__":
    install_requirements()

In [ ]:
import faiss
import numpy as np
import chromadb
from chromadb.utils import embedding_functions
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import json
import os
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
# Load pre-trained embedding model
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://www.sci.gov.in/judgements-case-no/"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Extract judgment links
for link in soup.find_all("a", href=True):
    if "pdf" in link["href"]:
        print("Found judgment PDF:", link["href"])


Found judgment PDF: https://judgments.ecourts.gov.in/pdfsearch/?p=pdf_search/index&escr_flag=Y
Found judgment PDF: http://cdnbbsr.s3waas.gov.in/s3ec0490f1f4972d133619a60c30f3559e/documents/misc/practice.pdf_0.pdf


In [ ]:
import fitz  # PyMuPDF
import requests

def extract_text_from_pdf(url):
    response = requests.get(url)
    pdf_path = "downloaded.pdf"

    # Save PDF locally
    with open(pdf_path, "wb") as f:
        f.write(response.content)

    # Extract text
    doc = fitz.open(pdf_path)
    text = "\n".join([page.get_text() for page in doc])

    return text

# Download and extract text
pdf_url = "http://cdnbbsr.s3waas.gov.in/s3ec0490f1f4972d133619a60c30f3559e/documents/misc/practice.pdf_0.pdf"
pdf_text = extract_text_from_pdf(pdf_url)

# Convert to JSON
pdf_data = {"url": pdf_url, "content": pdf_text}
json_output = json.dumps(pdf_data, indent=4)

# Save to file
with open("legal_documents.json", "w") as json_file:
    json_file.write(json_output)

print("✅ PDF data saved to `legal_documents.json`")


✅ PDF data saved to `legal_documents.json`


In [ ]:

# Load JSON file
with open("legal_documents.json", "r") as json_file:
    pdf_data = json.load(json_file)

# Print extracted content
print("✅ Extracted Data:\n")
print(json.dumps(pdf_data, indent=4, ensure_ascii=False))


✅ Extracted Data:

{
    "url": "http://cdnbbsr.s3waas.gov.in/s3ec0490f1f4972d133619a60c30f3559e/documents/misc/practice.pdf_0.pdf",
    "content": "SUPREME COURT OF INDIA\nHANDBOOK \nON\nPRACTICE AND PROCEDURE\nAND\n OFFICE PROCEDURE \n2017\n    \n\n\nCONTENTS\nPage Nos.\nChapter I - Preliminary\n1-4\n1.\nSeal of the Supreme Court\n1\n2. \nLanguage\n1\n3.\nDefinition\n1\n(i)\nAdvocate\n1\n(ii)\nAdvocate on-record\n1\n(iii)\nAppointed day\n1\n(iv)\nAllocated Matter\n1-2\n(v)\nChief Justice\n           2\n(vi)\nCode\n2\n(vii)\nConstitution\n2\n(viii)\nCourt and this Court\n      \n2\n(ix)\nCourt appealed from\n          2\n(x)\nCourt Fee\n2\n(xi)\nHigh Court\n2\n(xii)\nInterlocutory Application\n2\n(xiii)\nJudge\n          3\n(xiv)\nJudgment\n3\n(xv)\nMain Case\n3\n(xvi)\nMinor\n          3\n(xvii)\nMiscellaneous Application\n3\n(xviii)\nNot taken up case\n    \n3\n(xix)\nPrescribed\n3\n(xx)\nRecord\n3\n(xxi)\nRespondent\n3\n(xxii)\nThe Rules and Rules of the Court\n3\n(xxiii)\nSecretar

In [ ]:
pdf_data.keys()

dict_keys(['url', 'content'])

In [ ]:
# Set up Chrome options
chrome_options = Options()
chrome_options.binary_location = "/usr/bin/chromium-browser"  # Ensure correct path
chrome_options.add_argument("--headless")  # Run in background
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Set up WebDriver with the correct Chromedriver path
service = Service("/usr/bin/chromedriver")  # Ensure correct path
driver = webdriver.Chrome(service=service, options=chrome_options)


WebDriverException: Message: Service /usr/bin/chromedriver unexpectedly exited. Status code was: 1


In [ ]:
# Open E-Courts judgments page
url = "https://judgments.ecourts.gov.in/pdfsearch/?p=pdf_search/index&escr_flag=Y"
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Find all PDF links
pdf_links = []
elements = driver.find_elements(By.TAG_NAME, "a")
for elem in elements:
    href = elem.get_attribute("href")
    if href and "pdf" in href:
        pdf_links.append(href)

# Close browser
driver.quit()

# Save to JSON
data = {"judgments": pdf_links}
with open("ecourts_judgments.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

print("✅ Judgment links saved to `ecourts_judgments.json`")


NameError: name 'driver' is not defined